In [1]:
import random
import pickle
from util import *
import numpy as np
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors

In [2]:
#Step 1. read original word embedding
word_vecs = read_vector('../data/fasttext.txt')
embed_dim = len(word_vecs['a'])

In [3]:
#Step 2. read resnet image feature
img_feature = pickle.load(open('../img/img_mf.pkl', 'rb'))
img_feature.update(pickle.load(open('../img/img_mm.pkl', 'rb')))
img_feature.update(pickle.load(open('../img/img_ms.pkl', 'rb')))
img_feature.update(pickle.load(open('../img/img_sp.pkl', 'rb')))
img_feature.update(pickle.load(open('../img/img_yh.pkl', 'rb')))
img_feature.update(pickle.load(open('../img/img_fr.pkl', 'rb')))

In [4]:
#Step 3. read title image pair
fin = open('../data/trn_title_img.txt', 'r')
trn_data = [line.strip().split('\t') for line in fin.readlines()]
fin.close()
titles = [d[0] for d in trn_data]
img_names = [d[1] for d in trn_data]
print(len(titles))

566641


In [5]:
#Step 4. generate title embedding array
title_vecs = []
for t in titles:
    t_vec = sent2vec(t, word_vecs)
    title_vecs.append(t_vec)
title_vecs = np.array(title_vecs)

In [6]:
#Step 5. generate img feature array
img_vecs = []
for img_name in img_names:
    img_vecs.append(img_feature[img_name])
del img_feature
img_vecs = np.array(img_vecs)
img_dim = 1000

In [7]:
#Step 6. read test data, format: [label(0, 1), title 1, title 2]
test_data = read_test_data('../data/test_data.txt')
test_tv = []
test_label = []
for d in test_data:
    test_label.append(float(d[0]))
    test_tv.append(sent2vec(d[1], word_vecs))
    test_tv.append(sent2vec(d[2], word_vecs))

In [8]:
#Step 7. set hyper parameters
beta = 1e-2 #weight of orthogonal constraint
gamma = 1e-7 #weight of variational dropout
max_epoch = 10
batch_size = 128
steps = int(np.ceil(len(title_vecs)/batch_size))
split_index = 150 #so image-informative section index is (0:150), image-uninformative is (150:300)

In [9]:
#Step 8. build data generator
def generator():
    index = -1
    trn_idx = np.arange(len(img_vecs))
    random.shuffle(trn_idx)
    while True:
        if index < len(title_vecs)-1:
            index += 1
        else:
            index = 0
            trn_idx = np.arange(len(img_vecs))
            random.shuffle(trn_idx)
        yield (title_vecs[trn_idx[index]], img_vecs[trn_idx[index]])
        
data = tf.data.Dataset.from_generator(generator, (tf.float32, tf.float32),(tf.TensorShape(300), tf.TensorShape(1000)))
data = data.batch(batch_size)
iterator = data.make_one_shot_iterator()
batch_title_vecs, batch_img_vecs = iterator.get_next()

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [10]:
#Step 9. build model
#alpha stands for dropout rate in variational dropout
log_alpha = tf.get_variable("log_alpha", [embed_dim, 1], dtype=tf.float32, initializer=tf.initializers.random_normal(0,0.02))
#theta is the model to construct image feature
theta = tf.get_variable("theta", [embed_dim, img_dim], dtype=tf.float32, initializer=tf.initializers.random_normal(0,0.1))
#W is the transformation matrix
W = tf.get_variable("W", [embed_dim, embed_dim], initializer=tf.initializers.random_normal, dtype=tf.float32)

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
#orthogonal constraint
WtW = tf.matmul(W, W, transpose_a=True)
eye_matrix = tf.eye(embed_dim, dtype=tf.float32)
orth_loss = tf.losses.mean_squared_error(labels=eye_matrix, predictions=WtW)

#construct image with variational dropout
dup_w = tf.identity(W)
dis_title_vec = tf.matmul(dup_w, batch_title_vecs, transpose_b=True)
assign_op1 = log_alpha.assign(tf.clip_by_value(log_alpha, -10, 10))
head_drp = tf.identity(log_alpha[:split_index])
tail_drp = tf.clip_by_value(tf.identity(log_alpha[split_index:]), 2.5, 10)
assign_op2 = log_alpha.assign(tf.concat([head_drp, tail_drp],axis=0))
#Strong disentanglement: manually bound the dropout rates for Image-Uninformative
#Soft disentanglement: without control_dipendencies
with tf.control_dependencies([assign_op1, assign_op2]):
    alpha = tf.exp(log_alpha*0.5)
    epsilon = tf.random_normal(shape=alpha.shape)
    new_theta = theta*(1.0 + alpha*epsilon)
    pred_img = tf.matmul(dis_title_vec, new_theta, transpose_a=True)
    pred_loss = tf.losses.mean_squared_error(labels=batch_img_vecs, predictions=pred_img)

#calculate KL-divergence
k1, k2, k3 = 0.63576, 1.8732, 1.48695
kl = k1 * tf.sigmoid(k2 + k3 * log_alpha) - 0.5 * tf.log1p(tf.exp(-log_alpha))
kl_reg = - tf.reduce_sum(kl)

#weighted loss
losses = pred_loss + beta * orth_loss + gamma * kl_reg
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
train = optimizer.minimize(losses, var_list=[W, theta, log_alpha])

Instructions for updating:
Use tf.cast instead.


In [12]:
#Step 10. train and evaluate
with tf.Session() as sess:
    best_acc = 0
    tf.global_variables_initializer().run()
    for i in range(max_epoch):
        orth_losses = []
        pred_losses = []
        for j in range(steps):
            #beta = min(1.0 * i / 30, 1.0) #anther option is to gradually increase the weight of orthogonal constraint
            _, orth_loss_val, pred_loss_val = sess.run((train, orth_loss, pred_loss))
            orth_losses.append(orth_loss_val)
            pred_losses.append(pred_loss_val)
            print("\rEpoch {}/{}: {:.0%}".format(i+1, max_epoch, j/steps), end='')
        #validatioin and save the best result
        log_alpha_val, W_val = sess.run((log_alpha, W))
        dis_test_tv = np.matmul(W_val, np.array(test_tv).T).T
        val_acc = evaluate(W_val, dis_test_tv, test_label)
        print("\nPred loss: {:.5f}, Orth loss: {:.5f}, acc: {:.4f}".format(np.mean(pred_losses), np.mean(orth_losses), val_acc))
        if val_acc > best_acc:
            best_acc = val_acc
            dropout = np.exp(log_alpha_val)/(1+np.exp(log_alpha_val))
            np.save("../output/W_{}.npy".format(int(best_acc*10000)), W_val)
            np.save("../output/dropout_{}.npy".format(int(best_acc*10000)), dropout)
            print("Best result saved as W_{}.npy".format(int(best_acc*10000)))

Epoch 1/10: 100%
Pred loss: 64.18037, Orth loss: 469.45328, acc: 0.7260
Best result saved as W_7260.npy
Epoch 2/10: 100%
Pred loss: 0.00180, Orth loss: 228.39531, acc: 0.7256
Epoch 3/10: 100%
Pred loss: 0.00925, Orth loss: 49.89655, acc: 0.7240
Epoch 4/10: 100%
Pred loss: 0.00284, Orth loss: 4.72728, acc: 0.7292
Best result saved as W_7292.npy
Epoch 5/10: 100%
Pred loss: 0.79547, Orth loss: 0.48377, acc: 0.7336
Best result saved as W_7336.npy
Epoch 6/10: 100%
Pred loss: 0.00075, Orth loss: 0.45453, acc: 0.7344
Best result saved as W_7344.npy
Epoch 7/10: 100%
Pred loss: 0.00092, Orth loss: 0.36854, acc: 0.7376
Best result saved as W_7376.npy
Epoch 8/10: 100%
Pred loss: 0.00101, Orth loss: 0.14602, acc: 0.7500
Best result saved as W_7500.npy
Epoch 9/10: 100%
Pred loss: 0.66359, Orth loss: 0.02882, acc: 0.7500
Epoch 10/10: 100%
Pred loss: 0.00116, Orth loss: 0.02608, acc: 0.7512
Best result saved as W_7512.npy


In [13]:
#Step 11. check dropout rate
dropout = np.load("../output/dropout_7512.npy")
print(np.mean(dropout[:150]))
print(np.mean(dropout[150:]))

0.37191665
0.92415243


In [14]:
#Step 12. save evaluation result
W_val = np.load("../output/W_7512.npy")
dis_test_tv = np.matmul(W_val, np.array(test_tv).T).T
acc = evaluate(W_val, dis_test_tv, test_label, printout=True, savepath='../output/eval')

Best T: 0.69
Accuracy: 75.12%
Precision: 76.26%
Recall: 79.48%
F1-score: 77.83%


In [15]:
#evaluate Image-Informative
acc_d1 = evaluate(W_val, dis_test_tv[:, :split_index], test_label, printout=True, savepath='../output/eval_d1')

Best T: 0.70
Accuracy: 73.48%
Precision: 75.27%
Recall: 77.07%
F1-score: 76.16%


In [16]:
#evaluate Image-Uninformative
acc_d2 = evaluate(W_val, dis_test_tv[:, split_index:], test_label, printout=True, savepath='../output/eval_d2')

Best T: 0.68
Accuracy: 75.24%
Precision: 75.73%
Recall: 80.86%
F1-score: 78.21%


In [17]:
#Step 13. search for nearest neighbors
trans_word_vecs = {}
for w in word_vecs:
    wv = np.matmul(W_val, np.array(word_vecs[w]).T).T
    trans_word_vecs[w] = wv
dis_title_vecs = np.matmul(W_val, title_vecs.T).T
vecs = [dis_title_vecs, dis_title_vecs[:, :split_index], dis_title_vecs[:, split_index:]]
nbrs = []
for v in vecs:
    nbr = NearestNeighbors(n_neighbors=16, metric='cosine').fit(v)
    nbrs.append(nbr)
def search(key_words):
    v = sent2vec(key_words, trans_word_vecs)
    _, idxs0 = nbrs[0].kneighbors([v])
    _, idxs1 = nbrs[1].kneighbors([v[:split_index]])
    _, idxs2 = nbrs[2].kneighbors([v[split_index:]])
    diff1 = set(idxs1[0]).difference(set(idxs2[0]))
    diff2 = set(idxs2[0]).difference(set(idxs1[0]))
    print('====DisFastText====')
    for i in idxs0[0]:
        print(titles[i])
    print('\n')
    print('====DisFastText Image-Imformative====')
    for i in diff1:
        print(titles[i])
    print('\n')
    print('====DisFastText Image-Uinformative====')
    for i in diff2:
        print(titles[i])
    print('\n')

In [18]:
#sample some titles to search
for i in range(100):
    print(random.choice(titles))

【clinique 倩碧】水磁場72h超循環保濕凝膠(50ml)
黏樂趣 nelo 卡通造形重複貼掛勾組(駝羊)
【andzen】日系風格香氛負離子水氧機az-1168(來自澳洲單方精油10mlx3瓶)
【samsung】galaxy j5 / grand prime g530/g531/g530y專用 原廠電池(裸裝)
lee 牛仔褲 726中腰舒適刷色小直筒牛仔褲/ur 男款 深藍色
大象生活館 附發票 大家源1.5l美食養生鍋tcy-2745快煮壺/電茶壺/中藥壺/藥膳壺/花茶壺/優格機/美食鍋/溫酒
epson l3110 三合一 連續供墨複合機
【維維樂】舒必克 蜂膠兒童喉片-葡萄 （30顆/盒） 6盒 專為兒童
【大霹靂】samsung galaxy s8+ / s8 plus 布袋戲彩繪磁力皮套(紅塵雪)
【ihouse】sgs 防潮抗蟲蛀緩衝塑鋼加高五門三抽半開放式置物鞋櫃 寬97深33.5高180cm
adata威剛 sd600 512gb(紅或黑) usb3.1 外接式ssd行動硬碟
【阿舍食堂】外省乾麵「油蔥」(5包入475g)x4袋
【太極數位】cy hp gk200 有線機械式電競鍵盤 輕巧 方便 耐用
for nikon en-el11/12 智慧型充電器(micro usb輸入充電)
[106 美國直購] bissell 1095 清新芳香片(8入)spring breeze steam mop fragrance discs
日本threeway桌上型12位計算機 twc-20
【尚朋堂】3人份養生不鏽鋼電鍋(不鏽鋼配件)(ssc-007)
push! 戶外休閒登山用品195g高強度航空碳纖維coolmax手腕帶3ls三節式登山杖(2入)p67
【元山】智慧冰溫熱落地式飲水機ys-8211rwsab
新竹【超人3c】kinyo 耐嘉 us-301 2.0多媒體音箱 us301 金屬鐵網 桌上型電腦/筆電/手機/平板
【登記送dc扇+好禮3選1★富士通】4.5坪優級m系列r32變頻冷專分離式(ascg028cmtb/aocg028cmtb)
【day&day】雙層置物架-窄版(st3268-2s)
air cell-05 韓國7cm雙鉤型減壓背帶(背包專用)
【shiseido 資生堂東京櫃】怡麗絲爾極奢潤膠原柔膚水 170ml〈百貨公

In [21]:
search('英國vs沙宣 1200w國際電壓摺疊式負離子吹風機 vs912piw')

====DisFastText====
英國vs沙宣 1200w國際電壓摺疊式負離子吹風機 vs912piw
英國vs 沙宣1200w摺疊式吹風機 vs908pw
【英國vs沙宣 x 法國babyliss】25mm鈦金陶瓷電棒捲+1200w負離子國際電壓吹風機(美髮超值組)
【英國vs沙宣 x 法國babyliss】25mm鈦金陶瓷電棒捲+1200w負離子國際電壓吹風機(美髮超值組)
【英國vs沙宣 x 法國babyliss】25mm鈦金陶瓷電棒捲+1200w負離子國際電壓吹風機(美髮超值組)
【英國vs沙宣 x 法國babyliss】25mm鈦金陶瓷電棒捲+1200w負離子國際電壓吹風機(美髮超值組)
英國vs 沙宣1300w陶瓷摺疊吹風機 vs157rdrw
英國vs沙宣 1300w陶瓷摺疊吹風機 vs157rdrw
【英國vs沙宣】1300w陶瓷摺疊吹風機 vs157rdrw
【滿額贈.湊單再折】英國vs沙宣1200w摺疊式負離子深層滋潤吹風機(vs912piw 國際電壓)
【滿額贈.湊單再折】英國vs沙宣1200w摺疊式負離子深層滋潤吹風機(vs912piw 國際電壓)
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw


====DisFastText Image-Imformative====
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
達新牌 負離子吹風機 國際電壓 摺疊式(fd-2)
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
【滿額贈.湊單再折】英國vs沙宣1200w摺疊式負離子深層滋潤吹風機(vs912piw 國際電壓)
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw
vs沙宣 環球電摺疊負離子吹風機vs912piw【愛買】
vs沙宣 環球電摺疊負離子吹風機vs912piw【愛買】
英國vs沙宣 復刻粉漾負離子折疊式吹風機 vs590piw

In [22]:
search('【naturehike】繽紛撞色款雙面可戴空頂遮陽帽/防曬帽(三色任選)')

====DisFastText====
【naturehike】繽紛撞色款雙面可戴空頂遮陽帽/防曬帽(三色任選)
【naturehike】繽紛撞色款雙面可戴空頂遮陽帽/防曬帽(三色任選)
【naturehike】繽紛撞色款雙面可戴空頂遮陽帽/防曬帽 (紅粉色)
【naturehike】繽紛撞色款雙面可戴空頂遮陽帽/防曬帽 (藍紫色)
【naturehike】迷彩潮流款速乾透氣漁夫帽/遮陽帽/防曬帽 (五色任選)
【naturehike】迷彩潮流款速乾透氣漁夫帽/遮陽帽/防曬帽(五色任選)
【naturehike】迷彩潮流款速乾透氣漁夫帽/遮陽帽/防曬帽(五色任選)
【fifi飛時尚】兩用造型蕾絲布面遮陽空頂帽 戶外防曬帽(8色任選)
【cute ii lady】時尚亮片大帽檐空頂防曬遮陽帽(桃)
【cute ii lady】時尚亮片大帽檐空頂防曬遮陽帽(桃)
《條紋帽》條紋雙色雙面大帽沿遮陽帽 雙面可戴 帽沿可折 mz0668 防曬帽 漁夫帽 很輕巧
《條紋帽》條紋雙色雙面大帽沿遮陽帽 雙面可戴 帽沿可折 mz0668 防曬帽 漁夫帽 很輕巧
【cute ii lady】日本抗uv時尚涼感帽(8色任選)
【naturehike】upf50+時尚款折疊速乾鴨舌帽/遮陽帽/防曬帽(粉色)
【naturehike】upf50+時尚款折疊速乾鴨舌帽/遮陽帽/防曬帽(粉色)
【naturehike】upf50+時尚款折疊速乾鴨舌帽/遮陽帽/防曬帽(粉色)


====DisFastText Image-Imformative====
【cute ii lady】時尚亮片大帽檐空頂防曬遮陽帽(粉)
【cute ii lady】時尚亮片大帽檐空頂防曬遮陽帽(粉)
【naturehike】upf50+時尚款折疊速乾鴨舌帽/遮陽帽/防曬帽(粉色)
【lorensa蘿芮】抗uv蝴蝶結純棉透氣純色可折防風大帽簷防曬遮陽帽(卡其)
【naturehike】upf50+時尚款折疊速乾鴨舌帽/遮陽帽/防曬帽(粉色)
《條紋帽》條紋雙色雙面大帽沿遮陽帽 雙面可戴 帽沿可折 mz0668 防曬帽 漁夫帽 很輕巧
【naturehike】upf50+時尚款折疊速乾鴨舌帽/遮陽帽/防曬帽(粉色)
《條紋帽》條紋雙色雙面大帽沿遮陽帽 雙面可戴 帽沿可折 mz0668 防曬帽 漁夫帽 很輕巧
【natur